In [ ]:
#!pip install selenium
#!pip install chromium-chromedriver

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from selenium import webdriver
import time
import re
import os
import json
import boto3
from botocore.exceptions import ClientError
import logging
import pandas as pd

from selenium.webdriver.support.ui import WebDriverWait #hasta que los elementos carguen 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support import expected_conditions as EC #condicion esperada

In [ ]:
def upload_file(file_name, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    client = boto3.client(
                    service_name='s3'
                    )
    try:
        
        #print(json.dumps(object_name))
        
        with open("data_file.json", "w") as write_file:
            json.dump(object_name, write_file)
            
        #response = client.put_object(Bucket='data-scraping-anuncios',
        #    Key="{}".format(file_name),
        #    Body=json.dumps(object_name)
                                     
        #)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [28]:
ruta_ = "C:\\Users\\diego.quintero\\Downloads\\chromedriver.exe"


url_principal = 'https://www.milanuncios.com'
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--start-maximized')
chrome_options.add_argument('--disable-extensions')


chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--window-size=1280x1696')
chrome_options.add_argument('--user-data-dir=/tmp/user-data')
chrome_options.add_argument('--hide-scrollbars')
chrome_options.add_argument('--enable-logging')
chrome_options.add_argument('--log-level=0')
chrome_options.add_argument('--v=99')
chrome_options.add_argument('--single-process')
chrome_options.add_argument('--data-path=/tmp/data-path')
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--homedir=/tmp')
chrome_options.add_argument('--disk-cache-dir=/tmp/cache-dir')
chrome_options.add_argument('user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36')


#ruta_ = "C:\\Users\\Usuario\\Documents\\chromedriver.exe"
ruta_ = "C:\\Users\\diego.quintero\\Downloads\\chromedriver.exe"

driver = webdriver.Chrome(ruta_, chrome_options=chrome_options)


driver.get(url_principal)
captcha = driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[1]/div/div/div/footer/div/button[2]')
captcha.click()
# time.sleep(1)

search = driver.find_element_by_xpath(
    '//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div/div/div[2]/div[1]/div/form/button/span')
search.click()
time.sleep(1)

try:
    captcha2 = driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[1]/div/div/div/footer/div/button[2]')
    captcha2.click()
except:
    pass


resultDict= []

#div.ma-LayoutBasic.div.ma-AdvertisementPageLayout.ma-AdvertisementPageLayout-justify--center > div.ma-AdvertisementPageLayout-left > div.ma-LayoutFullHeight > div.ma-LayoutBasic-content.ma-Listing > div.ma-LayoutBasicMainContent > main > nav > div > button.ma-ButtonBasic.ma-ButtonBasic--neutral.ma-NavigationPagination-nextButton.ma-ButtonBasic--xsmall

try: #Try para evitar excepciones por no encontrar elementos
    
    for i in range(200):
        for scroll in range(6):
        #driver.execute_script("window.scrollBy(0,3000)", "")
            driver.execute_script("window.scrollBy(0,3000)", "")
            time.sleep(1)
            principals_links = driver.find_elements_by_xpath(
        '//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/div[2]/article['+str(i)+']/div/div[1]/div/h2/a')
        secundary_links = driver.find_elements_by_xpath(
        '//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/div[2]/article['+str(i)+']/div/div[2]/div[1]/div[1]/div[1]/a')
        descriptions = driver.find_elements_by_xpath(
        '//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/div[2]/article['+str(i)+']/div/div[2]/div[1]/div[1]/div[2]/p')
 
    #Patrón Xpath precio
    #//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/div[2]/article[13]/div/div[2]/div[2]/div/div/p/span
    #//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/div[2]/article[17]/div/div[2]/div[2]/div/div/p/span
    #<span class="ma-AdPrice-value ma-AdPrice-value--default ma-AdPrice-value--heading--m">442 €</span>
        price = driver.find_elements_by_xpath(
        '//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/div[2]/article['+str(i)+']/div/div[2]/div[2]/div/div/p/span')
        location_link = driver.find_elements_by_xpath( 
        '//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/div[2]/article['+str(i)+']/div/div[2]/div[1]/div[1]/div[1]/p/span')        
        type_car = driver.find_elements_by_xpath( 
        '//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/div[2]/article['+str(i)+']/div/div[2]/div[2]/div/ul/li/span/span')

    #patron oferta/demanda
        demand_of = driver.find_elements_by_xpath( 
        '//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/div[2]/article['+str(i)+']/header/p')

    #Patron año del carro    
        anno_vehicle = driver.find_elements_by_xpath(
            '//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/div[2]/article['+str(i)+']/div/div[2]/div[2]/div/ul/li[1]/span/span')
    
    #Color del vehículo
        color_vehicle = driver.find_elements_by_xpath(
        '//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/div[2]/article[18]/div/div[2]/div[2]/div/ul/li[3]/span/span')

        advice = dict()
    
        for ind, link in enumerate(principals_links):
            principal = link.get_attribute('href')
            subcategory = secundary_links[ind].get_attribute('href')
            advice['name'] = re.sub('-', ' ', re.findall('\.com/.*[a-z]/(.*)-[0-9].*\.htm', principal)[0])
            advice['description'] = descriptions[ind].text
            advice['category'] = re.sub('-', ' ', re.findall('com/(.*)/', principal)[0])
            advice['subcategory'] = re.sub('-', ' ', re.findall('com/(.*)/', subcategory)[0])        
            advice['link'] = principal
            advice['location'] = location_link[ind].text
            advice['Oferta_Demanda'] = demand_of[ind].text
            try:
                advice['Price'] = price[ind].text
            except:
                advice['Price']= ""
                continue
            try:
                advice['Kms/Type_vehicle'] = type_car[ind].text 
            except:
                advice['Kms/Type_vehicle'] = ""
                continue
            try:
                advice['Other_skills'] = color_vehicle[ind].text
            except:
                advice['Other_skills'] = ""
                continue
            try:
                advice['Anno_Auto'] = anno_vehicle[ind].text
            except:
                advice['Anno_Auto'] = ""
                continue
                         
        resultDict.append(advice)
                
        #print("i-->", i," scroll--> ", scroll)
    try:
        #paginator = driver.find_element_by_xpath('//nav/button['+i+']')
        
        paginator = driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/nav/div/button['+(i+1)+']')
        
        print('try paginator: ', paginator, i)
        paginator.click()
        
    except:
        time.sleep(2)
        #paginator = driver.find_element_by_xpath('//nav/button['+i+']')
        paginator = driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div[1]/div[2]/div[2]/div[1]/main/nav/div/button['+(i+1)+']')
        print('except paginator:', paginator, i)
        paginator.click()
        
    time.sleep(1)

        
except:#Fin Try para evitar excepciones por no encontrar elementos
    pass

driver.close()

In [29]:
coche_list = pd.DataFrame(resultDict)

In [30]:
coche_list.head()

,name,description,category,subcategory,link,location,Oferta_Demanda,Price,Kms/Type_vehicle,Other_skills,Anno_Auto
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,compro coches con embargo,vehículos con deudas EMBARGADOS Reserva domini...,coches de segunda mano,coches de segunda mano en sevilla,https://www.milanuncios.com/coches-de-segunda-...,Sevilla,DEMANDA,8.000 €,Automático,81 m²,Automático
2,compro coches,Se compran toda clase de vehículos: autocarava...,coches de segunda mano,coches de segunda mano en malaga,https://www.milanuncios.com/coches-de-segunda-...,Málaga,DEMANDA,20.000 €,Automático,81 m²,Automático
3,compro todo tipo de coches,"Compramos todo tipo de coches, 4x4, pick up, f...",coches de segunda mano,coches de segunda mano en madrid,https://www.milanuncios.com/coches-de-segunda-...,Madrid,DEMANDA,9.000 €,Automático,81 m²,Automático
4,compramos vehiculos con embargo precinto,COMPRAMOS COCHES .FURGONETAS.4X4 TODO-TERRENOS...,todoterreno de ocasion,todoterreno de ocasion en sevilla,https://www.milanuncios.com/todoterreno-de-oca...,Sevilla,DEMANDA,,NaN,NaN,NaN


In [27]:
#coche_list.to_csv(r'C:/Users/Usuario/.spyder-py3/coches_precios.csv', index=None, header=True, encoding='utf-8-sig')
coche_list.to_csv(r'C:/Users/diego.quintero/Downloads/coches_precios.csv', index=None, header=True, encoding='utf-8-sig')